In [6]:
import dash
from dash import html, Input, Output

app = dash.Dash(__name__)


toggler_on = {'background-color': 'black',
               'color': 'white',
               'height': '30px',
               'width': '100px',
               'margin-top': '20px',  # Reduced padding
               'margin-left': '20px',  # Reduced padding
               'margin-right': '-15px',
               'border-radius': '10px',  # Smooth edges
               'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.2)',  # Floating effect
               'border': 'none',  # Remove border               
               'position': 'relative',
               'z-index': '1'}

toggler_off = {'background-color': 'white',
               'color': 'black',
               'height': '30px',
               'width': '100px',
               'margin-top': '20px',  # Reduced padding
               'margin-left': '20px',  # Reduced padding
               'margin-right': '-15px',
               'border-radius': '10px',  # Smooth edges
               'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.2)',  # Floating effect
               'border': 'none',  # Remove border               
               'position': 'relative',
               'z-index': '1'}

button_2024 = {'background-color': '#c00000',
               'color': 'white',
               'height': '30px',
               'width': '50px',
               'margin-top': '20px',  # Reduced padding
               'margin-left': '20px',  # Reduced padding
               'margin-right': '-15px',
               'border-radius': '10px',  # Smooth edges
               'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.2)',  # Floating effect
               'border': 'none',  # Remove border               
               'position': 'relative',
               'z-index': '1'}

button_2023 = {'background-color': '#e97132',
               'color': 'white',
               'height': '30px',
               'width': '50px',
               'margin-top': '20px',  # Reduced padding
               'margin-left': '20px',  # Reduced padding
               'margin-right': '-15px',               
               'border-radius': '10px',  # Smooth edges
               'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.2)',  # Floating effect
               'border': 'none',  # Remove border               
               'position': 'relative',
               'z-index': '1'}

button_2022 = {'background-color': '#bfbec4',
               'color': 'white',
               'height': '30px',
               'width': '50px',
               'margin-top': '20px',  # Reduced padding
               'margin-left': '20px',  # Reduced padding
               'border-radius': '10px',  # Smooth edges
               'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.2)',  # Floating effect
               'border': 'none',  # Remove border               
               'position': 'relative',
               'z-index': '1'}

off_button = {'background-color': 'white',
              'color': 'grey',
              'height': '30px',
              'width': '50px',
              'margin-top': '20px',  # Reduced padding
              'margin-left': '20px',  # Reduced padding
               'margin-right': '-15px',                             
              'border-radius': '10px',  # Smooth edges
              'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.2)',  # Floating effect
               'border': 'none',  # Remove border              
              'position': 'relative',
              'z-index': '1'}

app.layout = html.Div([
    
    
    html.Button(id='chart_toggle',
                children=['chart_toggle'],
                n_clicks=0,
                style=toggler_on
    ),
    html.Button(id='range_toggle',
                children=['range_toggle'],
                n_clicks=0,
                style=toggler_on
    ),
    
    html.Button(id='2024_toggle',
                children=['2024'],
                n_clicks=0,
                style=button_2024
    ),
    html.Button(id='2023_toggle',
                children=['2023'],
                n_clicks=0,
                style=button_2023
    ),
    html.Button(id='2022_toggle',
                children=['2022'],
                n_clicks=0,
                style=button_2022
    ),
])

@app.callback(
    Output('chart_toggle', 'style'), 
    Output('chart_toggle', 'children'),     
    [Input('chart_toggle', 'n_clicks')]
)
def change_chart_button_style(n_clicks):
    if n_clicks % 2 == 1:
        return toggler_off, 'Line'
    else:
        return toggler_on, 'Seasonality'

@app.callback(
    Output('range_toggle', 'style'), 
    Output('range_toggle', 'children'),     
    [Input('range_toggle', 'n_clicks')]
)
def change_chart_button_style(n_clicks):
    if n_clicks % 2 == 1:
        return toggler_off, '5 year'
    else:
        return toggler_on, '2015-2019'

@app.callback(
    Output('2024_toggle', 'style'), 
    [Input('2024_toggle', 'n_clicks')]
)
def change_2024_button_style(n_clicks):
    if n_clicks % 2 == 1:
        return off_button
    else:
        return button_2024

@app.callback(
    Output('2023_toggle', 'style'), 
    [Input('2023_toggle', 'n_clicks')]
)
def change_2023_button_style(n_clicks):
    if n_clicks % 2 == 1:
        return off_button
    else:
        return button_2023

@app.callback(
    Output('2022_toggle', 'style'), 
    [Input('2022_toggle', 'n_clicks')]
)
def change_2022_button_style(n_clicks):
    if n_clicks % 2 == 1:
        return off_button
    else:
        return button_2022

if __name__ == '__main__':
    app.run_server(debug=True,port=8051)
